In [3]:
# Data Handling & Manipulation
import pandas as pd                                         # for dataframes, CSV/Excel reading, tabular data manipulation
import numpy as np                                          # for numerical operations and array handling

# Data Visualisation (Static)
import matplotlib.pyplot as plt                             # for creating static plots
from matplotlib.ticker import FuncFormatter                 # for customising tick labels (e.g., currency, %)
import seaborn as sns                                       # for statistical visualisation (heatmaps, distplots, etc.)

# Data Visualisation (Interactive)
import plotly.express as px                                 # for quick and interactive visualisation
import plotly.graph_objects as go                           # for custom interactive plots
from plotly.subplots import make_subplots                   # for interactive subplots

# Statistical Testing & Inference
from statsmodels.stats.proportion import proportions_ztest  # for comparing proportions (e.g., late vs. on-time)
from scipy.stats import (
    normaltest,                                             # for checking normality
    chi2_contingency,                                       # for categorical association
    mannwhitneyu,                                           # for non-parametric testing
    ttest_ind,                                              # for independent sample t-test
    f_oneway,                                               # for one-way ANOVA
    kruskal,                                                # for Kruskal-Wallis test
    kstest,                                                 # for Kolmogorov-Smirnov test
    spearmanr                                               # for spearmean correlation
)
import statsmodels.api as sm                                # for advanced statistical modelling and diagnostics
import statsmodels.formula.api as smf                       # for formula-based statistical models
from statsmodels.stats.multicomp import pairwise_tukeyhsd   # for post-hoc tests after ANOVA

# Data Quality & Missing Value Visualisation
import missingno as msno                                    # for visualising missing data patterns

# System & Settings
import os                                                   # for file handling and directory operations
import warnings                                             # to suppress or manage warning messages
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_colwidth', None)                 # display full content in cells (useful for text data)

In [4]:
# List of date columns for each Olist dataset
# This dictionary maps each dataset filename to a list of columns that should be parsed as dates.
date_cols = {
    'olist_orders_dataset.csv': [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
    ],
    'olist_order_items_dataset.csv': [
        'shipping_limit_date',
    ],
    'olist_order_reviews_dataset.csv': [
        'review_creation_date',
        'review_answer_timestamp',
    ],
    # The following datasets have NO date columns:
    # 'olist_customers_dataset.csv'
    # 'olist_geolocation_dataset.csv'
    # 'olist_order_payments_dataset.csv'
    # 'olist_products_dataset.csv'
    # 'olist_sellers_dataset.csv'
    # 'product_category_name_translation.csv'
    'master_olist_dataset.csv': [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
        'shipping_limit_date',
        'review_creation_date',
        'review_answer_timestamp',
    ]
}

def read_olist_csv(path):
    """
    Reads an Olist CSV and parses dates for the correct columns.
    Args:
        path (str): Path to the CSV file.
    Returns:
        pd.DataFrame: Loaded dataframe with date columns parsed as datetime.
    """
    # Extract just the filename, e.g., 'olist_orders_dataset.csv'
    filename = os.path.basename(path)
    # Get the correct date columns for this file, or an empty list
    parse_dates = date_cols.get(filename, [])
    # Read the CSV, parsing the specified date columns (if any)
    return pd.read_csv(path, parse_dates=parse_dates)

Load master dataset

In [5]:
master_olist_dataset = read_olist_csv('../data/cleaned_data/master_olist_dataset.csv')

In [6]:
master_olist_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115093 entries, 0 to 115092
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       115093 non-null  object        
 1   customer_id                    115093 non-null  object        
 2   order_status                   115093 non-null  object        
 3   order_purchase_timestamp       115093 non-null  datetime64[ns]
 4   order_approved_at              115093 non-null  datetime64[ns]
 5   order_delivered_carrier_date   115093 non-null  datetime64[ns]
 6   order_delivered_customer_date  115093 non-null  datetime64[ns]
 7   order_estimated_delivery_date  115093 non-null  datetime64[ns]
 8   customer_unique_id             115093 non-null  object        
 9   customer_zip_code_prefix       115093 non-null  int64         
 10  customer_city                  115093 non-null  object        
 11  

In [9]:
master_olist_ml_df = master_olist_dataset.copy()

In [10]:
master_olist_ml_df['is_late']    = master_olist_dataset['order_delivered_customer_date'] > master_olist_dataset['order_estimated_delivery_date']

Drop unused features

In [11]:
features_after_approval = [
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'review_id',
    'review_score',
    'review_comment_title',
    'review_comment_message',
    'review_creation_date',
    'review_answer_timestamp',
    'has_review',
]

In [12]:
master_olist_ml_df = master_olist_ml_df.drop(columns=features_after_approval)

master_olist_ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115093 entries, 0 to 115092
Data columns (total 33 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       115093 non-null  object        
 1   customer_id                    115093 non-null  object        
 2   order_status                   115093 non-null  object        
 3   order_purchase_timestamp       115093 non-null  datetime64[ns]
 4   order_approved_at              115093 non-null  datetime64[ns]
 5   order_estimated_delivery_date  115093 non-null  datetime64[ns]
 6   customer_unique_id             115093 non-null  object        
 7   customer_zip_code_prefix       115093 non-null  int64         
 8   customer_city                  115093 non-null  object        
 9   customer_state                 115093 non-null  object        
 10  order_item_id                  115093 non-null  float64       
 11  

# Features Creation

## Geographical Features

In [13]:
cleaned_df_geolocation      = read_olist_csv('../data/cleaned_data/olist_geolocation_dataset.csv')

### Add aggregated lat & lng pair for both customers & sellers

In [14]:
# Merge location data
# Get median geo coordinates for each zip code
median_coords = cleaned_df_geolocation.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'median',
    'geolocation_lng': 'median'
}).reset_index()

# Merge customer geo data
master_olist_ml_df = master_olist_ml_df.merge(
    median_coords,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
).rename(columns={
    'geolocation_lat': 'customer_lat',
    'geolocation_lng': 'customer_lng'
})

# Merge seller geo data 
master_olist_ml_df = master_olist_ml_df.merge(
    median_coords,
    left_on='seller_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left',
    suffixes=('_drop', '')
).rename(columns={
    'geolocation_lat': 'seller_lat',
    'geolocation_lng': 'seller_lng'
})

# Drop redundant columns
master_olist_ml_df = master_olist_ml_df.drop(columns=['geolocation_zip_code_prefix', 'geolocation_zip_code_prefix_drop'], errors='ignore')
master_olist_ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115093 entries, 0 to 115092
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       115093 non-null  object        
 1   customer_id                    115093 non-null  object        
 2   order_status                   115093 non-null  object        
 3   order_purchase_timestamp       115093 non-null  datetime64[ns]
 4   order_approved_at              115093 non-null  datetime64[ns]
 5   order_estimated_delivery_date  115093 non-null  datetime64[ns]
 6   customer_unique_id             115093 non-null  object        
 7   customer_zip_code_prefix       115093 non-null  int64         
 8   customer_city                  115093 non-null  object        
 9   customer_state                 115093 non-null  object        
 10  order_item_id                  115093 non-null  float64       
 11  

#### Handle missing lat/lng

The missing lat & lng are due to customer/seller having a zip_code_prefix that doesn't exists in the geolocation dataset

In [ ]:
# Check missing percentage for customer and seller coordinates
missing_coords = {
    'customer_lat': (master_olist_ml_df['customer_lat'].isna().sum() / len(master_olist_ml_df)) * 100,
    'customer_lng': (master_olist_ml_df['customer_lng'].isna().sum() / len(master_olist_ml_df)) * 100,
    'seller_lat': (master_olist_ml_df['seller_lat'].isna().sum() / len(master_olist_ml_df)) * 100,
    'seller_lng': (master_olist_ml_df['seller_lng'].isna().sum() / len(master_olist_ml_df)) * 100
}

# Create a DataFrame to display the missing percentages
missing_coords_df = pd.DataFrame({
    'Column': list(missing_coords.keys()),
    'Missing %': list(missing_coords.values())
})

# Calculate the number of rows with any missing coordinate
rows_with_missing = master_olist_ml_df[
    master_olist_ml_df['customer_lat'].isna() | 
    master_olist_ml_df['customer_lng'].isna() | 
    master_olist_ml_df['seller_lat'].isna() | 
    master_olist_ml_df['seller_lng'].isna()
].shape[0]

# Calculate percentage of rows with any missing coordinate
pct_rows_with_missing = (rows_with_missing / len(master_olist_ml_df)) * 100

# Display results
print(missing_coords_df)
print(f"\nRows with any missing coordinate: {rows_with_missing} ({pct_rows_with_missing:.2f}%)")

         Column  Missing %
0  customer_lat   0.261528
1  customer_lng   0.261528
2    seller_lat   0.225035
3    seller_lng   0.225035

Rows with any missing coordinate: 559 (0.49%)


In [22]:
# overall share of orders by state
overall_customer = (master_olist_ml_df['customer_state']
           .value_counts(normalize=True)
           .rename('overall_customer_prop'))

overall_seller = (master_olist_ml_df['seller_state']
          .value_counts(normalize=True)
          .rename('overall_seller_prop'))

# Create masks for missing coordinates
missing_customer_mask = master_olist_ml_df['customer_lat'].isna()
missing_seller_mask = master_olist_ml_df['seller_lat'].isna()

# Calculate proportion for missing customer coordinates by state
missing_customer_prop = (master_olist_ml_df.loc[missing_customer_mask, 'customer_state']
                .value_counts(normalize=True)
                .rename('missing_customer_prop'))

# Calculate proportion for missing seller coordinates by state
missing_seller_prop = (master_olist_ml_df.loc[missing_seller_mask, 'seller_state']
                .value_counts(normalize=True)
                .rename('missing_seller_prop'))

# Create a dataframe for customer bias analysis
customer_bias = (pd.concat([overall_customer, missing_customer_prop], axis=1)
                .fillna(0)
                .assign(customer_lift=lambda d: d['missing_customer_prop'] / d['overall_customer_prop']))

# Create a dataframe for seller bias analysis
seller_bias = (pd.concat([overall_seller, missing_seller_prop], axis=1)
                .fillna(0)
                .assign(seller_lift=lambda d: d['missing_seller_prop'] / d['overall_seller_prop']))

# Show the top 10 states with highest lift for customers
display(customer_bias.sort_values('customer_lift', ascending=False))

display(seller_bias.sort_values('seller_lift', ascending=False))

,overall_customer_prop,missing_customer_prop,customer_lift
customer_state,,,
DF,0.021113,0.617940,29.267733
RO,0.002442,0.006645,2.721486
PI,0.004770,0.009967,2.089447
CE,0.013033,0.026578,2.039300
GO,0.020653,0.029900,1.447757
MA,0.007099,0.009967,1.404047
RN,0.004883,0.006645,1.360743
PB,0.005422,0.006645,1.225541
BA,0.034181,0.039867,1.166351


,overall_seller_prop,missing_seller_prop,seller_lift
seller_state,,,
DF,0.008080,0.158301,19.590705
MG,0.078354,0.258687,3.301518
SP,0.713440,0.552124,0.773889
RS,0.019541,0.007722,0.395175
PR,0.076599,0.023166,0.302433
CE,0.000860,0.000000,0.000000
PA,0.000070,0.000000,0.000000
SE,0.000087,0.000000,0.000000
PI,0.000096,0.000000,0.000000


Only 0.5 % of orders lacked ZIP-level coordinates. Because 62 % of these came from Brasília (a compact metro area), we imputed missing lat/lng with the median of all geolocation points in the same state and flagged those rows. This retains all data while avoiding external APIs and introducing no target leakage.

- Only 0.26 % of customers and 0.23 % of sellers lack coordinates.
- \> 60 % of those gaps are in DF (Brasília)—a single, geographically small metro area.
- Remaining gaps are scattered, each contributing ≪ 0.05 % of total rows.	
- A single median point for DF is basically a city centroid (< 15 km error)—more precise than the entire state centroid would be in SP or MG.
- Errors elsewhere (SP, MG, etc.) affect < 0.1 % of rows, so any blurring is statistically negligible.

In [24]:
# build once
state_centroids = (cleaned_df_geolocation
                   .groupby('geolocation_state', as_index=False)
                   .agg(state_lat=('geolocation_lat', 'median'),
                        state_lng=('geolocation_lng', 'median')))

def impute_state_centroid(df, role):
    df = df.merge(state_centroids,
                  left_on=f'{role}_state',
                  right_on='geolocation_state',
                  how='left')
    mask = df[f'{role}_lat'].isna()
    df.loc[mask, f'{role}_lat'] = df.loc[mask, 'state_lat']
    df.loc[mask, f'{role}_lng'] = df.loc[mask, 'state_lng']
    df[f'{role}_coord_imputed'] = mask          # flag uncertainty
    return df.drop(columns=['state_lat', 'state_lng', 'geolocation_state'])

master_olist_ml_df = impute_state_centroid(master_olist_ml_df, 'customer')
master_olist_ml_df = impute_state_centroid(master_olist_ml_df, 'seller')

master_olist_ml_df = master_olist_ml_df.drop(columns=['customer_coord_imputed', 'seller_coord_imputed'], errors='ignore')

master_olist_ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115093 entries, 0 to 115092
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       115093 non-null  object        
 1   customer_id                    115093 non-null  object        
 2   order_status                   115093 non-null  object        
 3   order_purchase_timestamp       115093 non-null  datetime64[ns]
 4   order_approved_at              115093 non-null  datetime64[ns]
 5   order_estimated_delivery_date  115093 non-null  datetime64[ns]
 6   customer_unique_id             115093 non-null  object        
 7   customer_zip_code_prefix       115093 non-null  int64         
 8   customer_city                  115093 non-null  object        
 9   customer_state                 115093 non-null  object        
 10  order_item_id                  115093 non-null  float64       
 11  

### Add customer-seller distance using Haversine Distance (kilometers)

In [25]:
# Calculate distance between customer and seller
# Using Haversine formula to calculate distance between two coordinates
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

# Apply the formula to calculate distance for all rows with valid coordinates
mask = (~master_olist_ml_df['customer_lat'].isna()) & (~master_olist_ml_df['seller_lat'].isna())
master_olist_ml_df.loc[mask, 'distance_km'] = haversine(
    master_olist_ml_df.loc[mask, 'customer_lat'],
    master_olist_ml_df.loc[mask, 'customer_lng'],
    master_olist_ml_df.loc[mask, 'seller_lat'],
    master_olist_ml_df.loc[mask, 'seller_lng']
)

# Print summary statistics of the new distance feature
print(f"Distance statistics (km):")
print(f"Mean: {master_olist_ml_df['distance_km'].mean():.2f}")
print(f"Median: {master_olist_ml_df['distance_km'].median():.2f}")
print(f"Min: {master_olist_ml_df['distance_km'].min():.2f}")
print(f"Max: {master_olist_ml_df['distance_km'].max():.2f}")
print(f"Null values: {master_olist_ml_df['distance_km'].isna().sum()}")

Distance statistics (km):
Mean: 597.16
Median: 433.64
Min: 0.00
Max: 3398.55
Null values: 0
